In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [10]:
import re
import os
import warnings
import datetime

import pandas as pd
import numpy as np
import pandasql as psql

from solution import *
from datetime import timedelta
pd.set_option("display.max_rows", None, "display.max_columns", None) # Let pandas print the entire dataframe


In [1]:
import os
import itertools
import pandas as pd

def get_data(timestamp, input_file):
    timestamp = pd.Timestamp(timestamp)
    data = pd.read_csv(input_file)
    data['time'] = data['time'].apply(
        lambda dt: dt[:-5] + '00:00'
    )
    data['time'] = pd.to_datetime(data['time'])
    data = data.loc[data.time <= timestamp]

    return data


def fill_nan(data):
    data = data.groupby(
        ['device', 'time']
    )['device_activated'].agg(
        'max'
    ).reset_index()

    data = data.set_index(
        ['time', 'device']
    ).unstack(
        fill_value=0
    ).asfreq(
        'H', fill_value=0
    ).stack().sort_index(
        level=1
    ).reset_index()

    return data


def extract_feature(data):
    data['day_name'] = data.time.dt.day_name()
    data['day'] = data.time.dt.day
    data['week'] = data.time.dt.isocalendar().week
    data['hour'] = data.time.dt.hour

    data = data[[
        'device',
        'day_name',
        'hour',
        'device_activated',
        'time'
    ]]
    return data


def data_pipeline(timestamp, input_file):
    data = get_data(timestamp, input_file)
    data = fill_nan(data)
    data = extract_feature(data)

    return data


def get_pred_hash(timestamp, input_file, output_file):
    data = data_pipeline(timestamp, input_file)
    timestamp = pd.Timestamp(timestamp)
    train_data = data[data.time <= timestamp]

    pred_hash = train_data.groupby(
        ['device', 'day_name', 'hour']
    ).agg(
        device_activated=('device_activated', 'max')
    ).reset_index()

    return pred_hash


def generate_prediction_holder(timestamp):
    timestamp = pd.Timestamp(timestamp)
    split_point = timestamp.round(freq='T')
    next_24_hours = pd.date_range(split_point, periods=24, freq='H').ceil('H')
    device_names = ['device_' + str(i) for i in range(1,8)]

    xproduct = list(itertools.product(next_24_hours, device_names))
    prediction_holder = pd.DataFrame(xproduct, columns=['time', 'device'])

    prediction_holder['day_name'] = prediction_holder.time.dt.day_name()
    prediction_holder['hour'] = prediction_holder.time.dt.hour
    columns = [
        'time',
        'device',
        'day_name',
        'hour'
    ]

    prediction_holder = prediction_holder[columns]

    return prediction_holder


def make_pred(timestamp, input_file, output_file):
    pred_hash = get_pred_hash(timestamp, input_file, output_file)
    prediction_holder = generate_prediction_holder(timestamp,pred_hash)
    pred = prediction_holder.merge(pred_hash, how='left', on=['device', 'day_name', 'hour'])
    pred = pred[[
        'time',
        'device',
        'device_activated'
    ]]

    return pred

In [17]:
timestamp = '2016-07-04 23:59:59'
input_file = 'data\device_activations.csv'
output_file = 'data\myresult.csv'

prediction_holder = generate_prediction_holder(timestamp)

pred_hash = get_pred_hash(timestamp, input_file, output_file)
pred = prediction_holder.merge(pred_hash, how='left', on=['device', 'day_name', 'hour'])
pred = pred.fillna(0)

In [18]:
prediction_holder.shape, pred_hash.shape,pred.shape

((168, 4), (528, 4), (168, 5))

In [19]:
pred_hash

,device,day_name,hour,device_activated
0,device_2,Friday,4,0
1,device_2,Friday,5,0
2,device_2,Friday,6,1
3,device_2,Friday,7,1
4,device_2,Friday,8,1
5,device_2,Friday,9,1
6,device_2,Friday,10,1
7,device_2,Friday,11,1
8,device_2,Friday,12,1
9,device_2,Friday,13,1


In [20]:
pred

,time,device,day_name,hour,device_activated
0,2016-07-05 00:00:00,device_1,Tuesday,0,0.0
1,2016-07-05 00:00:00,device_2,Tuesday,0,0.0
2,2016-07-05 00:00:00,device_3,Tuesday,0,0.0
3,2016-07-05 00:00:00,device_4,Tuesday,0,0.0
4,2016-07-05 00:00:00,device_5,Tuesday,0,0.0
5,2016-07-05 00:00:00,device_6,Tuesday,0,0.0
6,2016-07-05 00:00:00,device_7,Tuesday,0,0.0
7,2016-07-05 01:00:00,device_1,Tuesday,1,0.0
8,2016-07-05 01:00:00,device_2,Tuesday,1,0.0
9,2016-07-05 01:00:00,device_3,Tuesday,1,0.0
